<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_EM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Limpieza Empleo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install unidecode

import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/EM'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

In [ ]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

In [ ]:
# # CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
# CU_EM = pd.read_excel('/content/CU_EM.xlsx')
# TO_EM = pd.read_excel('/content/TO_EM.xlsx')
# AB_EM = pd.read_excel('/content/AB_EM.xlsx')
# # CR_EM = pd.read_excel('/content/CR_EM.xlsx')

In [ ]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'FECHA DE NACIMIENTO', 'FECHA NACIM.']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'C. POSTAL': 'CP',
        'COD. POSTAL': 'CP',
        'TIPO PROTESIS ': 'PROTESIS',
        'TIPO DE PROTESIS': 'PROTESIS',
        'FECHA NACIM.': 'FECHA NACIMIENTO',
        'FECHA NAC.': 'FECHA NACIMIENTO',
        'GRADO DISCAP.': 'GRADO DISCAPACIDAD',
        '%DISCAP.': 'GRADO DISCAPACIDAD',
        '%PERDIDA': 'GRADO PERDIDA',
        'SIST. COMUNIC.': 'SISTEMA COMUNICACION',
        'G. DEPENDENCIA': 'GRADO DEPENDENCIA',
        'GRADO DE PERDIDA': 'GRADO PERDIDA',
        'FECHA INSERCCION': 'FECHA ATENCION',
        'ANO DE AT.': 'YEAR ATENCION',
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def CP_year(df):
    if 'FECHA NACIMIENTO' in df.columns:
      df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

      df['FECHA NACIMIENTO'] = df['FECHA NACIMIENTO'].apply(lambda x: np.nan if str(x).isdigit() or x == '(ADULTO)' else x)
      df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

      # Convertir a numérico, dejando como NaN los valores que no sean convertibles
      df['CP'] = pd.to_numeric(df['CP'], errors='coerce').astype('Int64')

      df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')
      CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()
      df = df[~CP_loc_filter]
    return df

def other_columns(df):
  if 'PROTESIS' in df.columns:
    df['AUD O PROTESIS'] = df['PROTESIS'].apply(lambda x: 'NO' if x in [np.nan, '-','NINGUNA', 'NO'] else 'SI')
    df.drop('PROTESIS', axis=1, inplace=True)

  if 'SISTEMA COMUNICACION' in df.columns:
    df['SISTEMA COMUNICACION'] = df['SISTEMA COMUNICACION'].apply(lambda x: 'ORAL' if x in ['ORAL', 'ORAL/LSE', 'LSE/ORAL', 'COM. ORAL', 'COM.ORAL'] else x)
    df['SISTEMA COMUNICACION'] = df['SISTEMA COMUNICACION'].apply(lambda x: 'LSE' if x in ['L.S.E', 'L.S.E.', 'LS', 'LSE'] else x)

  if 'GRADO DISCAPACIDAD' in df.columns:
    df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].apply(lambda x: x / 100 if x > 1 else x)
    df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].fillna(0)

  if 'LOCALIDAD' in df.columns:
    # Limpiar los datos eliminando contenido entre paréntesis
    df['LOCALIDAD'] = df['LOCALIDAD'].str.replace(r'\s*\([^)]*\)', '', regex=True)

  if 'GENERO' in df.columns:
    df['GENERO'] = df['GENERO'].apply(lambda x: 'MUJER' if x in ['M', 'MUJER', 'FEMENINO'] else x)
    df['GENERO'] = df['GENERO'].apply(lambda x: 'HOMBRE' if x in ['H', 'HOMBRE', 'MASCULINO'] else x)

  if 'FECHA ATENCION' in df.columns:
    df['FECHA ATENCION'] = pd.to_datetime(df['FECHA ATENCION'], errors='coerce')
    df['YEAR ATENCION'] = df['FECHA ATENCION'].dt.year
    df.drop('FECHA ATENCION', axis=1, inplace=True)

  return df

def delete_not_important_columns(df):
  # Añado 'AÑO ATENCIÓN' para SAAF
  needed_columns = ['FECHA NACIMIENTO', 'LOCALIDAD', 'CP', 'YEAR NACIMIENTO']

  # Creación de la lista de columnas consideradas para posible eliminación
  social_columns = [col for col in df.columns if col not in needed_columns]

  # Porcentaje máximo de valores nulos permitido
  max_percentage = 1/3

  # Identifico columnas para eliminar
  del_columns = []
  for col in social_columns:
      if df[col].isnull().sum() / len(df) >= max_percentage:
          del_columns.append(col)

  # Eliminar las columnas identificadas
  df.drop(columns=del_columns, inplace=True)
  return df

In [ ]:
# Suponiendo que 'dict_of_dfs' es tu diccionario de DataFrames
for df_key, df in dic_dataframes.items():
    dic_dataframes[df_key] = transform_df(df)
    dic_dataframes[df_key] = mapping_names(df)
    dic_dataframes[df_key] = CP_year(df)
    dic_dataframes[df_key] = other_columns(df)
    dic_dataframes[df_key] = delete_not_important_columns(df)

In [ ]:
# # CÓDIGO DE USO PARA EL CURRO
# CU_EM = transform_df(CU_EM)
# CU_EM = mapping_names(CU_EM)
# CU_EM = CP_year(CU_EM)
# CU_EM = other_columns(CU_EM)
# CU_EM = delete_not_important_columns(CU_EM)

# TO_EM = transform_df(TO_EM)
# TO_EM = mapping_names(TO_EM)
# TO_EM = CP_year(TO_EM)
# TO_EM = other_columns(TO_EM)
# TO_EM = delete_not_important_columns(TO_EM)

# AB_EM = transform_df(AB_EM)
# AB_EM = mapping_names(AB_EM)
# AB_EM = CP_year(AB_EM)
# AB_EM = other_columns(AB_EM)
# AB_EM = delete_not_important_columns(AB_EM)

In [ ]:
AB_EM = dic_dataframes.get("AB_EM.xlsx")
CR_EM = dic_dataframes.get("CR_EM.xlsx")
CU_EM = dic_dataframes.get("CU_EM.xlsx")
TO_EM = dic_dataframes.get("TO_EM.xlsx")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   YEAR ATENCION         31 non-null     int64         
 1   FECHA NACIMIENTO      30 non-null     datetime64[ns]
 2   GENERO                31 non-null     object        
 3   LOCALIDAD             31 non-null     object        
 4   CP                    31 non-null     Int64         
 5   GRADO DISCAPACIDAD    31 non-null     float64       
 6   SISTEMA COMUNICACION  31 non-null     object        
 7   YEAR NACIMIENTO       30 non-null     float64       
 8   AUD O PROTESIS        31 non-null     object        
dtypes: Int64(1), datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 2.3+ KB


In [ ]:
CLM_EM = pd.concat([AB_EM, CR_EM, CU_EM, TO_EM])

In [ ]:
# List of DataFrames
dfs = [AB_EM, CR_EM, CU_EM, TO_EM, CLM_EM]

# Corresponding folder paths on Google Drive
folder_paths = [
    '/content/drive/My Drive/PFG_FASPAS/AB',
    '/content/drive/My Drive/PFG_FASPAS/CR',
    '/content/drive/My Drive/PFG_FASPAS/CU',
    '/content/drive/My Drive/PFG_FASPAS/TO',
    '/content/drive/My Drive/PFG_FASPAS/CLM'
]

# Corresponding file names
file_names = ['AB_EM_limpio.xlsx', 'CR_EM_limpio.xlsx', 'CU_EM_limpio.xlsx', 'TO_EM_limpio.xlsx', 'CLM_EM_limpio.xlsx']

<class 'pandas.core.frame.DataFrame'>
Index: 116 entries, 0 to 69
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR ATENCION            46 non-null     float64       
 1   FECHA NACIMIENTO         112 non-null    datetime64[ns]
 2   GENERO                   116 non-null    object        
 3   LOCALIDAD                116 non-null    object        
 4   CP                       116 non-null    Int64         
 5   GRADO DISCAPACIDAD       116 non-null    float64       
 6   SISTEMA COMUNICACION     101 non-null    object        
 7   YEAR NACIMIENTO          112 non-null    float64       
 8   AUD O PROTESIS           116 non-null    object        
 9   SISTEMA DE COMUNICACION  15 non-null     object        
 10  GRADO PERDIDA            15 non-null     object        
dtypes: Int64(1), datetime64[ns](1), float64(3), object(6)
memory usage: 11.0+ KB


In [ ]:
# Iterate over the DataFrames, folder paths, and file names
for df, folder_path, file_name in zip(dfs, folder_paths, file_names):
    # Define the complete file path
    file_path = f"{folder_path}/{file_name}"

    # Save the DataFrame as an Excel file
    df.to_excel(file_path, index=False)

print("All DataFrames have been exported successfully.")